In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
import torch

# 全局GPU设备（若之前已计算协同组，此处可复用设备）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ----------------------
# 1. 加载预计算的协同组数据（需提前生成）
# ----------------------
def load_correlated_group(cycle_days=5):
    """加载预计算的周期协同组数据（之前GPU代码生成的CSV）"""
    group_path = f"./gpu_high_cycle_groups_{cycle_days}d.csv"  # 协同组文件路径
    group_df = pd.read_csv(group_path, encoding="utf-8-sig")
    
    # 找到目标股票（600570）所在的协同组
    target_stock = "600570.XSHG"
    target_group_row = None
    for _, row in group_df.iterrows():
        if target_stock in row["股票代码"]:
            target_group_row = row
            break
    
    if target_group_row is None:
        raise ValueError(f"未找到{target_stock}在{cycle_days}日周期的协同组")
    
    # 解析协同组股票列表
    correlated_stocks = target_group_row["股票代码"].split(",")
    print(f"加载{cycle_days}日周期协同组：共{len(correlated_stocks)}只股票，包含{target_stock}")
    return correlated_stocks, target_group_row["平均协同得分"]

# ----------------------
# 2. 数据加载（适配Backtrader，含目标股票+协同组股票）
# ----------------------
def load_strategy_data(correlated_stocks, data_dir, cycle_days=5):
    """
    加载策略所需数据：目标股票（600570）+ 协同组其他股票的日K数据
    返回：Backtrader数据列表（datas[0]为目标股票，datas[1:]为协同组股票）
    """
    target_stock = "600570.XSHG"
    bt_datas = []
    
    # 批量加载协同组内所有股票的日K数据
    for stock_code in correlated_stocks:
        # 读取股票日K数据（假设数据格式：date, open, high, low, close, volume, pre_close）
        stock_path = f"{data_dir}/stock_daily_price_{stock_code.replace('.', '_')}.parquet"  # 需根据实际路径调整
        try:
            df = pd.read_parquet(stock_path)
        except FileNotFoundError:
            print(f"警告：{stock_code}数据缺失，跳过")
            continue
        
        # 数据预处理
        df["date"] = pd.to_datetime(df["date"])
        df.set_index("date", inplace=True)
        df = df[["open", "high", "low", "close", "volume", "pre_close"]]  # 适配Backtrader格式
        df = df.dropna()
        
        # 计算5日周期特征（用于策略信号）
        df["5d_cycle_return"] = df["close"].rolling(window=cycle_days).apply(
            lambda x: x.iloc[-1] / x.iloc[0] - 1, raw=False
        )
        df["5d_ma"] = df["close"].rolling(window=cycle_days).mean()  # 5日周期均线（支撑位）
        
        # 转换为Backtrader数据格式
        bt_data = bt.feeds.PandasData(
            dataname=df,
            fromdate=df.index.min(),
            todate=df.index.max(),
            timeframe=bt.TimeFrame.Days,  # 日K数据
            compression=1
        )
        bt_data._name = stock_code  # 标记股票代码
        bt_datas.append(bt_data)
    
    # 确保目标股票在数据列表首位（datas[0]）
    target_idx = next((i for i, data in enumerate(bt_datas) if data._name == target_stock), None)
    if target_idx is None:
        raise ValueError(f"目标股票{target_stock}数据缺失，无法运行策略")
    # 交换位置，将目标股票移至首位
    bt_datas[0], bt_datas[target_idx] = bt_datas[target_idx], bt_datas[0]
    
    print(f"数据加载完成：共{len(bt_datas)}只股票（1只目标+{len(bt_datas)-1}只协同组股票）")
    return bt_datas

# ----------------------
# 3. 协同组共振趋势策略（Backtrader核心）
# ----------------------
class CorrelatedTrendStrategy(bt.Strategy):
    params = (
        ("cycle_days", 5),          # 周期天数（与协同组周期一致）
        ("resonance_ratio", 0.6),   # 协同组共振阈值（≥60%股票趋势向上）
        ("stop_loss_ratio", 0.03),  # 止损比例（跌破5日均线3%）
        ("hold_max_days", 10),      # 最大持仓天数（避免趋势钝化）
    )

    def __init__(self):
        # 数据引用：datas[0] = 目标股票（600570），datas[1:] = 协同组其他股票
        self.target_data = self.datas[0]
        self.correlated_datas = self.datas[1:]
        
        # 目标股票核心指标（从数据中获取预计算的周期特征，或实时计算）
        self.target_close = self.target_data.close
        self.target_5d_return = self.target_data.d5d_cycle_return  # 5日周期涨跌幅
        self.target_5d_ma = self.target_data.d5d_ma                # 5日周期均线
        
        # 交易状态变量
        self.in_position = False       # 是否持仓
        self.buy_price = 0             # 买入价格
        self.hold_days = 0             # 持仓天数
        self.last_trade_date = None    # 跨日重置标记

    def calculate_group_resonance(self):
        """计算协同组共振度：趋势向上的股票占比"""
        up_count = 0
        valid_count = 0
        for data in self.correlated_datas:
            # 跳过无周期数据的股票
            if np.isnan(data.d5d_cycle_return[0]):
                continue
            # 统计5日周期涨跌幅>0的股票（趋势向上）
            if data.d5d_cycle_return[0] > 0:
                up_count += 1
            valid_count += 1
        # 避免除以0（无有效协同股票时，默认共振度0）
        return up_count / valid_count if valid_count > 0 else 0

    def next(self):
        current_date = self.target_data.datetime.date(0)
        # 跨日重置持仓天数
        if current_date != self.last_trade_date:
            if self.in_position:
                self.hold_days += 1
            self.last_trade_date = current_date

        # 1. 计算核心信号指标
        group_resonance = self.calculate_group_resonance()  # 协同组共振度
        target_trend_up = self.target_5d_return[0] > 0      # 目标股票自身趋势
        target_above_ma = self.target_close[0] > self.target_5d_ma[0]  # 收盘价在5日均线上

        # 2. 离场条件（持仓时判断）
        if self.in_position:
            # 条件1：目标股票趋势反转（5日周期涨跌幅≤0）
            # 条件2：协同组共振消失（≤40%股票趋势向上）
            # 条件3：跌破5日均线止损（收盘价<5日均线×(1-止损比例)）
            # 条件4：持仓超最大天数（避免长期横盘）
            stop_condition = (
                (not target_trend_up) or
                (group_resonance <= 0.4) or
                (self.target_close[0] < self.target_5d_ma[0] * (1 - self.p.stop_loss_ratio)) or
                (self.hold_days >= self.p.hold_max_days)
            )
            if stop_condition:
                self.sell(data=self.target_data, size=self.position.size)
                self.in_position = False
                self.hold_days = 0
                print(f"离场：{current_date}，价格{self.target_close[0]:.2f}，共振度{group_resonance:.2f}")
            return

        # 3. 进场条件（无持仓时判断）
        # 条件1：目标股票趋势向上+在5日均线上（自身趋势有效）
        # 条件2：协同组共振度≥60%（组内趋势一致）
        enter_condition = target_trend_up and target_above_ma and (group_resonance >= self.p.resonance_ratio)
        if enter_condition:
            # 仓位管理：投入总资金的80%（避免满仓风险）
            buy_size = int(self.broker.getvalue() * 0.8 / self.target_close[0])
            self.buy(data=self.target_data, size=buy_size)
            self.in_position = True
            self.buy_price = self.target_close[0]
            print(f"进场：{current_date}，价格{self.target_close[0]:.2f}，共振度{group_resonance:.2f}")

# ----------------------
# 4. 策略回测执行
# ----------------------
if __name__ == "__main__":
    # 配置参数
    cycle_days = 5  # 周期天数（需与预计算的协同组周期一致）
    data_dir = r"D:\workspace\xiaoyao\data\stock_daily_price"  # 日K数据目录
    initial_cash = 1000000  # 初始资金100万

    # 步骤1：加载预计算的协同组
    correlated_stocks, avg_corr = load_correlated_group(cycle_days=cycle_days)

    # 步骤2：加载策略所需的日K数据（目标股票+协同组股票）
    bt_datas = load_strategy_data(correlated_stocks, data_dir, cycle_days=cycle_days)

    # 步骤3：初始化Backtrader引擎
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(initial_cash)  # 设置初始资金

    # 步骤4：添加数据（所有协同组股票数据）
    for data in bt_datas:
        cerebro.adddata(data)

    # 步骤5：配置交易成本（佣金0.02%，印花税0.1%）
    cerebro.broker.setcommission(commission=0.0002, stamp_duty=0.001)  # 卖出时收印花税
    cerebro.broker.set_slippage_fixed(0.0005)  # 滑点0.05%

    # 步骤6：添加策略
    cerebro.addstrategy(CorrelatedTrendStrategy, cycle_days=cycle_days)

    # 步骤7：添加绩效分析指标
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Days, riskfreerate=0.02)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

    # 步骤8：运行回测
    print(f"\n回测开始，初始资金：{initial_cash:.2f}元，周期：{cycle_days}日")
    results = cerebro.run()
    strat = results[0]

    # 步骤9：输出回测结果
    final_cash = cerebro.broker.getvalue()
    print(f"\n回测结束，最终资金：{final_cash:.2f}元，总收益：{(final_cash - initial_cash)/initial_cash*100:.2f}%")
    
    # 关键绩效指标
    returns_analysis = strat.analyzers.returns.get_analysis()
    print(f"年化收益率：{returns_analysis.get('rnorm100', 0):.2f}%")
    
    sharpe = strat.analyzers.sharpe.get_analysis().get('sharperatio', None)
    print(f"夏普比率：{sharpe:.2f}" if sharpe is not None else "夏普比率：无法计算")
    
    print(f"最大回撤：{strat.analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")
    
    # 交易细节
    trade_analysis = strat.analyzers.trade_analyzer.get_analysis()
    if 'total' in trade_analysis and trade_analysis['total']['closed'] > 0:
        total_trades = trade_analysis['total']['closed']
        won_trades = trade_analysis['won']['total']
        print(f"总交易次数：{total_trades}")
        print(f"胜率：{won_trades/total_trades*100:.2f}%")
        print(f"平均单次收益：{trade_analysis['won']['pnl']['average']:.2f}元")
        print(f"平均单次亏损：{trade_analysis['lost']['pnl']['average']:.2f}元")

SyntaxError: invalid syntax (<ipython-input-1-3c4104305757>, line 105)